In [5]:
!pip install pyautogen
!pip install openai
!pip install python-dotenv
!pip install playwright
!pip install beautifulsoup4
!pip install nest-asyncio
!pip install ipywidgets


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libopus.so.0                                     ║
║     libwebpdemux.so.2                                ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libhyphen.so.0                                   ║
║     libflite.so.1                                    ║
║     libflite_usenglish.so.1                          ║
║     libflite_cmu_grapheme_lang.so.1                  ║
║     libflite_cmu_grapheme_lex.so.1                   ║
║     libflite_cmu_indic_lang.so.1                     ║
║     libflite_cmu_indic_lex.so.1                      ║
║     libflite_cmulex.so.1                             ║
║     libflite_cmu_time_awb.so.1                   

In [1]:
from dotenv import load_dotenv
import os
from autogen import UserProxyAgent, AssistantAgent
from openai import AzureOpenAI
from typing import Annotated
import httpx
import json
import subprocess

load_dotenv(override=True)

llm_config = {
    "model": "gpt-4o",
    "api_type": "azure",
    "api_key": "38b2b9b8286c4123b3bf51cc07111785",
    "base_url": "https://ik-oai-west-us.openai.azure.com/",
    "api_version": "2024-02-01"
}

client = AzureOpenAI(
    # This is the default and can be omitted
    api_key="38b2b9b8286c4123b3bf51cc07111785",
    api_version="2024-02-01",
    azure_endpoint = "https://ik-oai-west-us.openai.azure.com/"
)

In [4]:
lowel_prompt = """### Role
You are a research agent specialized in insurance underwriting. 

### Instruction
Your task is to act as a research agent specialized in providing detailed information for insurance underwriting purposes, focusing on cancer and other life-threatening diseases. 

### Requirements
You MUST cover the following aspects for each disease:
- Description
- Related terms (other names or diagnosis for the disease e.g. Myocardial infarction also named Heart attack)
- Disease classification
- Prevalence
- Cause
- Signs and symptoms
- Diagnosis and Investigations
- Treatment
- Prognosis (include remission and recurrence) & Complications
- **Information that helps assessing/underwriting the risk 
- Risk classification (e.g., stage, ocular MG, borderline hyperlipidaemia; familial hyperlipidaemia)
- Risk related to:
   · Life (mortality rate; impact on the life expectancy; loss of life expectancy; and the impact by the different classifications, e.g. controlled cases over uncontrolled cases)
   · Hospitalization (risk of hospitalization; hospitalization rate of the disease; average duration of each hospitalization; reason for hospitalization; and the impact by the different classifications, e.g. controlled cases over uncontrolled cases)
   · Disability (risk of total and permanent physical disability, risk of functional disability; and the impact by the different classifications, e.g. controlled cases over uncontrolled cases )
   · Cancer (risk of cancer; remission and recurrence rate; survival rate; and the impact by the different classifications, e.g. controlled cases over uncontrolled cases)
- References from reputable sources (WHO, government institutions, NGOs, clinics)
### Example
Provide the information in an easy-to-read markdown format, using bullet points and tables wherever possible.

### Context and Data 
Ensure that your answer is factual, unbiased, and avoids relying on stereotypes. Use only reputable sources for references.
You are encourage to use scrape_web to find as much detail as you can, as brave_search often give incomplete information.

### Sample Output Primer:
Here is the beginning of an example output for cancer to serve as a template:

```markdown
## Disease: Lung Cancer
### Related terms:  Myocardial infarction also named Heart attack

### Description
- **Lung cancer** is a type of cancer that begins in the lungs. The lungs are two spongy organs in your chest that take in oxygen when you inhale and release carbon dioxide when you exhale.

### Disease Classification
- **Small cell lung cancer (SCLC)**
- **Non-small cell lung cancer (NSCLC)**

### Prevalence
- Global Incidence: 2.2 million new cases annually (Source: WHO)
- Higher prevalence in smokers and older adults

### Cause
- Primary cause: smoking
- Other factors: exposure to radon gas, secondhand smoke, asbestos, and other carcinogens

### Signs and Symptoms
- Persistent cough
- Coughing up blood
- Shortness of breath
- Chest pain
- Hoarseness
- Weight loss

### Diagnosis and Investigations
- Imaging tests: Chest X-ray, CT scan
- Tissue sampling (biopsy)
- Molecular testing for specific gene mutations

### Treatment
- Surgery
- Radiation therapy
- Chemotherapy
- Targeted drug therapy
- Immunotherapy

### Prognosis (include remission and recurrence) & Complications
- **Prognosis:**
  - Early-stage: Higher likelihood of remission
  - Advanced-stage: Lower survival rates
- **Recurrence:**
  - Common in advanced stages
- **Complications:**
  - Metastasis to bones, brain, liver
  - Respiratory failure

### Risk Classification
- **Stage**: Based on TNM system
  - T: Size of the primary tumor
  - N: Extent of spread to lymph nodes
  - M: Presence of metastasis

### Mortality Risk – Life Expectancy
- Early-stage: 5-year survival rate ~60%
- Advanced-stage: 5-year survival rate ~15%

### Morbidity Risk
- Hospitalization risk: High, especially post-diagnosis
- Critical illness risk: Significant during treatment

### Co-morbidity Risk
- Higher in patients with existing respiratory conditions

### Information that helps assessing/underwriting the risk (e.g. age, gender, type of disease, severity of disease, presence of other risk factors, particular laboratory results; comorbidities)

### Risk related to:

· Life (mortality rate; impact on the life expectancy; loss of life expectancy; and the impact by the different classifications, e.g. controlled cases over uncontrolled cases)

· Hospitalization (risk of hospitalization; hospitalization rate of the disease; average duration of each hospitalization; reason for hospitalization; and the impact by the different classifications, e.g. controlled cases over uncontrolled cases)

· Disability (risk of total and permanent physical disability, risk of functional disability; and the impact by the different classifications, e.g. controlled cases over uncontrolled cases )

· Cancer (risk of cancer; remission and recurrence rate; survival rate; and the impact by the different classifications, e.g. controlled cases over uncontrolled cases)

### References
- WHO: [Link]
- National Cancer Institute: [Link]

---
<Additional diseases will follow a similar structure>
```"""

julius_prompt = """### Role
You are a research agent specialized in insurance underwriting. 

### Instruction
Your task is to research unfamiliar diseases upon request and provide concise, deeply detailed information from reputable sources. Ensure all information is accurate and properly cited. 
You are provided with tools to help complete your task. Search the web for relevant webpages, and when needed navigate the url for detailed information.

### Research Criteria:
1. **Introduction**: Provide a brief overview of the disease (approx. 100 words).
2. **Symptoms and Effects**: List the symptoms and explain how they affect the patient.
3. **Treatment and Costs**: Describe available treatment methods and their associated costs.
4. **Prognosis and Statistics**: Detail the prognosis of the disease, including survival and recovery statistics.
5. **Major Complications**: Highlight the major complications associated with the disease.
6. **Causes**: Explain the main causes of the disease.

### Response Requirements:
- Responses should be in bullet points for easy reading.
- Information should be deeply detailed.
- Cite the reputable and trustworthy sources used (Sources to be provided).
- Ensure that your answer is unbiased and avoids relying on stereotypes.

### Quality Control
- You are encourage to use scrape_web to ensure detailed and accurate information.

### Example:
**Disease Name: [Insert Disease Name]**

**Introduction:**
[Provide a brief overview of the disease (approx. 100 words).]

**Symptoms and Effects:**
- Symptom 1: Detailed description
- Symptom 2: Detailed description
- ...

**Treatment and Costs:**
- Treatment Method 1: Description and cost
- Treatment Method 2: Description and cost
- ...

**Prognosis and Statistics:**
- Prognosis details
- Survival rates: Specific statistics
- Recovery rates: Specific statistics

**Major Complications:**
- Complication 1: Detailed description
- Complication 2: Detailed description
- ...

**Causes:**
- Cause 1: Detailed description
- Cause 2: Detailed description
- ...

**Sources:**
- Source 1
- Source 2"""

In [3]:
from IPython.display import display, Markdown

language_country_map = {
    'en':'ALL',
    'jp':'JP'
}

async def research(input_text:str, language:str='en'):
    
    user_proxy_agent = UserProxyAgent(
        name='user_proxy_agent',
        is_termination_msg=lambda msg: msg.get('content', None) != None and msg.get('tool_calls', None) == None,
        code_execution_config=False,
        human_input_mode="NEVER",
        llm_config=llm_config
    )

    # created by FWD_AI_STUDIO

    uw_researcher_prompt = lowel_prompt
    
    uw_researcher = AssistantAgent(
        name = 'uw_researcher',
        system_message=uw_researcher_prompt,
        llm_config = llm_config,
        human_input_mode="NEVER",
    )

    # register the search tool for the uw_research
    # alternative sources are highly encouraged

    @user_proxy_agent.register_for_execution()
    @uw_researcher.register_for_llm(description="Search the web with Brave API to get information. Search for one concept at a time.")
    async def brave_search(query:Annotated[str, 'The search query. Could be a keyword, a question or a sentence.']) -> str:
        response = httpx.get(
            'https://api.search.brave.com/res/v1/web/search',
            params={
                'q':query,
                # 'search_lang':language,
                # 'country':language_country_map[language]
            },
            headers={
                'X-Subscription-Token':"BSAhYTht4n6HAm0NKA_emPGQPamY9MJ"
            }
        )

        results = response.json().get('web').get('results')[:1]
        results = [
            {k:result.get(k, None) for k in ['title', 'description', 'extra_snippets', 'url', 'page_age']}
            for result in results
        ]

        for result in results:
            url = result.get('url')
            output = subprocess.run(
                f'python3 scrape_web.py {url}',
                shell=True,
                capture_output=True,
            )
            result.update(
                {
                    'page_content':output.stdout.decode("utf-8")
                }
            )

        return json.dumps(results)

    language_prompt = ''
    if language == 'en':
        language_prompt = ''
    elif language == 'jp':
        language_prompt = ' 最終報告書を日本語で書く'
    
    res = await user_proxy_agent.a_initiate_chat(
        recipient=uw_researcher,
        clear_history=True,
        message = f'{input_text}{language_prompt}'
    )
    
    display(Markdown(res.chat_history[-1].get('content')))


In [7]:
topic = input("search term: ")
result = await research(topic, "jp")
print(result)

user_proxy_agent (to uw_researcher):

depression 最終報告書を日本語で書く

--------------------------------------------------------------------------------
uw_researcher (to user_proxy_agent):

***** Suggested tool call (call_VSNruvo1yW9oPLnZGL5pTMmk): brave_search *****
Arguments: 
{"query": "うつ病 説明 日本語"}
*****************************************************************************
***** Suggested tool call (call_tgWOR79G0NRD2glMZOAx8Kys): brave_search *****
Arguments: 
{"query": "うつ病 疾患分類"}
*****************************************************************************
***** Suggested tool call (call_j2ghYrjwWUnPouKKZ1sceKhV): brave_search *****
Arguments: 
{"query": "うつ病 有病率 日本"}
*****************************************************************************
***** Suggested tool call (call_zLx30azPtY7mqFOMld5baMXD): brave_search *****
Arguments: 
{"query": "うつ病 原因"}
*****************************************************************************
***** Suggested tool call (call_X4wPkDyuhctRpnqo5XQEFqDh

## 疾患: うつ病

### 説明
- **うつ病**は、持続的な気分の落ち込みや興味や喜びの喪失を特徴とする精神的な疾患です。脳内の神経伝達物質のバランスが崩れることが原因とされています。  
  [参考](https://kokoro.mhlw.go.jp/about-depression/ad001/)

### 疾患分類
- **大うつ病性障害**：持続的な強い抑うつ状態
- **うつ状態**：軽度から中等度の抑うつが持続

### 有病率
- 日本国内における生涯有病率は約6.6%  
  [参考](https://www.smilenavigator.jp/utsu/about/)

### 原因
- 生物学的要因（神経伝達物質の異常）
- 心理的要因（ストレス、トラウマ）
- 環境的要因（人間関係、仕事上の問題）

### 症状
- 持続的な憂うつな気分
- 興味または喜びの喪失
- 体重や食欲の変動
- 睡眠障害（不眠または過眠）
- 疲労感や倦怠感
- 自責の念または無価値感

### 診断と検査
- DSM-5診断基準に基づく精神科医の評価
- 抑うつ症状のスクリーニングテスト  

### 治療
- **薬物療法**：抗うつ薬（SSRI、SNRIなど）
- **心理療法**：認知行動療法（CBT）
- **その他治療法**：電気痙攣療法（ECT）、運動療法など

### 予後（寛解と再発）& 合併症
- **予後**：
  - 早期治療により寛解が期待できる
  - 再発率は高いとされる
- **合併症**：
  - 慢性化することがある
  - 自殺のリスクが高まる

### リスク分類
- **重症度**：
  - 軽度、中等度、重度
- **併存疾患**：他の精神疾患や身体疾患の有無

### 死亡リスク – 期待寿命
- 自殺のリスクから死亡リスクが増加
- 精神疾患の併存により身体的健康も悪化

### 罹患率 – 入院リスク; CIリスク
- **入院リスク**：
  - 重度のケースでは入院が必要になることがある
- **CIリスク**：
  - 持続的な抑うつ状態による生活機能の低下

### 併存疾患のリスク
- 不安障害、パニック障害などの併存リスクが高い

### 各ステージやシナリオにおける期待寿命
- 早期発見・早期治療：正常な寿命を期待できるが再発リスクは存在
- 治療が遅れる場合：再発リスクが高く、慢性化の可能性

### 入院リスクと率
- 重症例では入院治療が必要
- 再燃するたびに入院するリスク

### 引用元
- 厚生労働省: [ここのみみ](https://kokoro.mhlw.go.jp/about-depression/ad001/)
- 大塚製薬: [すまいのナビゲーター](https://www.smilenavigator.jp/utsu/about/)
- 慈友クリニック: [うつ病診断テスト](https://www.jiyu.or.jp/psychiatry/p-ambulatory/depression-test/)
- 東横会たわらクリニック: [治療と過ごし方](https://www.tawara-clinic.com/blog/002/)

---

他の疾患についても同じ形式でまとめます。

None
